## Emergency Detection

In [27]:
import pandas as pd
from patient_emergency_detection import emergency_detection_workflow, prepare_data_for_detection, detect_emergency
from base.minio_utils import load_model_from_minio
from base.influx_utils import fetch_all_sensor_data
from base.homecare_hub_utils import send_info, send_todo

In [28]:
sensor_data = fetch_all_sensor_data(
    start_hours=24,
    interval_hours=24
)
prepared_df = prepare_data_for_detection(sensor_data)
prepared_df.tail(5)

2025-01-03 19:00:21 [INFO] patient_emergency_detection - Starting data preparation for detection.
2025-01-03 19:00:21 [INFO] patient_emergency_detection - Original data shape: (1238, 5)
2025-01-03 19:00:21 [INFO] patient_emergency_detection - Handled 'sensor' as a list.
2025-01-03 19:00:21 [INFO] patient_emergency_detection - Timestamps converted to datetime.
2025-01-03 19:00:21 [INFO] patient_emergency_detection - Data sorted by timestamp.
2025-01-03 19:00:21 [INFO] patient_emergency_detection - Sensor names encoded successfully.
2025-01-03 19:00:21 [INFO] patient_emergency_detection - Mapped sensors to rooms.
2025-01-03 19:00:21 [INFO] patient_emergency_detection - Identified room-change points and assigned group IDs.
2025-01-03 19:00:21 [INFO] patient_emergency_detection - Calculated start and end times for each group.
2025-01-03 19:00:21 [INFO] patient_emergency_detection - Computed durations for each group.
2025-01-03 19:00:21 [INFO] patient_emergency_detection - Data preparation 

,group_id,room,start_time,end_time,duration,duration_seconds
76,77,bathroom,2025-01-03 11:33:54.773,2025-01-03 11:33:54.773,0 days 00:00:00,0.000
77,78,kitchen,2025-01-03 11:35:58.009,2025-01-03 12:18:42.269,0 days 00:42:44.260000,2564.260
78,79,bathroom,2025-01-03 12:21:18.244,2025-01-03 12:21:32.604,0 days 00:00:14.360000,14.360
79,80,kitchen,2025-01-03 12:27:02.348,2025-01-03 17:46:11.561,0 days 05:19:09.213000,19149.213
80,81,bathroom,2025-01-03 17:51:25.015,2025-01-03 17:59:55.860,0 days 00:08:30.845000,510.845


In [ ]:
room_stats = load_model_from_minio("occupancy")
room_stats

In [ ]:
_, message = emergency_detection_workflow()
print(message)

In [29]:
room = "livingroom"
duration = 240  # seconds
stats = pd.Series({"mean": 120, "std": 30})

_, result = detect_emergency(room, duration, stats)
print(result)

2025-01-03 19:00:24 [INFO] patient_emergency_detection - Room 'livingroom' stats - Mean: 120.00 seconds, Std: 30.00 seconds.
2025-01-03 19:00:24 [INFO] patient_emergency_detection - Duration bounds for emergency detection: (30.00, 210.00) seconds.
2025-01-03 19:00:24 [WARNING] patient_emergency_detection - 🚨 Emergency Alert! 🚨
 Room: livingroom,
 Patient has spent 0:04:00 here.
 Expected duration (mean): 0:02:00,
 Standard deviation (std): 0:00:30,
 Threshold used: 3,
 Allowed duration range: 0:00:30 - 0:03:30,
Duration is outside the allowed range!


🚨 Emergency Alert! 🚨
 Room: livingroom,
 Patient has spent 0:04:00 here.
 Expected duration (mean): 0:02:00,
 Standard deviation (std): 0:00:30,
 Threshold used: 3,
 Allowed duration range: 0:00:30 - 0:03:30,
Duration is outside the allowed range!


In [30]:
send_todo("🚨 Patient Emergency! 🚨", result, 2)

ToDo item saved successfully.
Response: None


## Motion Analysis

In [ ]:
from motion_analysis import (
    visualize_transitions_heatmap, 
    plot_bidirectional_transaction_graph, 
    summary_of_sleep_time_and_time_outside,
    count_daily_visits,
    detect_wake_up_times,
    detect_went_to_sleep_times,
    create_separate_heat_maps)
from base.minio_utils import load_model_from_minio
import pandas as pd
from base.homecare_hub_utils import send_info
from motion_analysis import analyse_motion_patterns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [ ]:
old_model = load_model_from_minio("motion", 2)
new_model = load_model_from_minio("motion", 1)

In [ ]:
old_model.head()

In [ ]:
plot_bidirectional_transaction_graph(old_model)

In [ ]:
plot_bidirectional_transaction_graph(new_model)

In [ ]:
visualize_transitions_heatmap(old_model)

#### Counting the sleep and went outside periods

In [ ]:
summary_of_sleep_time_and_time_outside(old_model)

In [ ]:
summary_of_sleep_time_and_time_outside(new_model)

#### Detect daliy trips to places

In [ ]:
count_daily_visits(new_model, transition_state='bathroom')

In [ ]:
count_daily_visits(new_model, transition_state='kitchen')

In [ ]:
count_daily_visits(new_model, transition_state='livingroombedarea')

In [ ]:
count_daily_visits(old_model, transition_state='livingroomdoor')

#### Wake up times

In [ ]:
detect_wake_up_times(new_model)

In [ ]:
detect_went_to_sleep_times(old_model)

In [ ]:
create_separate_heat_maps(new_model)

In [ ]:
message = analyse_motion_patterns()

In [ ]:
print(message)

In [26]:
from motion_analysis import analyse_motion_patterns
analyse_motion_patterns()

2025-01-03 18:59:31 [INFO] motion_analysis - 🔄 Loading old and new motion models from MinIO...
2025-01-03 18:59:31 [INFO] base.minio_utils - Initialized MinIO client successfully.
2025-01-03 18:59:31 [INFO] base.minio_utils - [MOTION MODEL] Loading model 'model_motion_20250103133818.json' (version 2).
2025-01-03 18:59:31 [INFO] base.minio_utils - [MOTION MODEL] 'model_motion_20250103133818.json' downloaded.
2025-01-03 18:59:31 [INFO] base.minio_utils - [MOTION MODEL] Model deserialized successfully.
2025-01-03 18:59:31 [INFO] base.minio_utils - Initialized MinIO client successfully.
2025-01-03 18:59:31 [INFO] base.minio_utils - [MOTION MODEL] Loading model 'model_motion_20250103133819.json' (version 1).
2025-01-03 18:59:31 [INFO] base.minio_utils - [MOTION MODEL] 'model_motion_20250103133819.json' downloaded.
2025-01-03 18:59:31 [INFO] base.minio_utils - [MOTION MODEL] Model deserialized successfully.
2025-01-03 18:59:31 [INFO] motion_analysis - ✅ Models loaded successfully.

2025-01-0

'📊 **📈 Motion Patterns Analysis Report 📉📊**\n\n📅 **Report Date:** 2025-01-03 17:59:31 UTC\n\n### 🕰️ **Old Motion Model** 🕰️\n\n#### 🛌 Sleep Time and 🌳 Time Outside Summary\n\n```      date day_of_week sleep_time time_outside\n2024-11-29      Friday          0       6h 16m\n2024-11-30    Saturday          0       5h 20m\n2024-12-01      Sunday     1h 45m       2h 28m\n2024-12-02      Monday     8h 28m      12h 44m\n2024-12-03     Tuesday     7h 11m       8h 57m\n2024-12-04   Wednesday     6h 54m       4h 50m\n2024-12-05    Thursday      4h 9m      11h 27m\n2024-12-06      Friday     1h 37m       1h 38m```\n\n#### 🚽 **Daily Visits to Bathroom**\n\n``` trip_date  bathroom_trip_count\n2024-12-02                    2\n2024-12-03                   18\n2024-12-04                   10\n2024-12-05                   14\n2024-12-06                   15```\n\n#### 🚽 **Daily Visits to Kitchen**\n\n``` trip_date  kitchen_trip_count\n2024-11-29                  11\n2024-11-30                  23\n202

## Burglary Detection

In [15]:
from burglary_detection import BurglaryDetector
from burglary_detection import create_burglary_message
from motion_model import train_motion_model
import pandas as pd
from base.homecare_hub_utils import send_todo
from base import base_logger

In [16]:
detector = BurglaryDetector(contamination=0.01, model_type='burglary')
detector.load_model(version=1)

Model version 1 loaded from MinIO successfully.


In [20]:
motion_model = train_motion_model(start_hours=24*7, interval_hours=24*7, time_threshold_seconds=1800)
detection = detector.detect(motion_model)

In [21]:
detection

(                  from                 to              leave_time  \
 0             bathroom  livingroombedarea 2025-01-01 20:40:00.413   
 1    livingroombedarea       went outside 2025-01-01 22:04:35.178   
 2         went outside           bathroom 2025-01-01 23:27:11.502   
 3             bathroom      went to sleep 2025-01-02 00:42:30.393   
 4        went to sleep            kitchen 2025-01-02 01:21:35.170   
 ..                 ...                ...                     ...   
 148            kitchen           bathroom 2025-01-03 11:31:46.201   
 149           bathroom            kitchen 2025-01-03 11:33:54.773   
 150            kitchen           bathroom 2025-01-03 12:18:42.269   
 151           bathroom            kitchen 2025-01-03 12:21:32.604   
 152            kitchen           bathroom 2025-01-03 17:46:11.561   
 
                  enter_time  duration  leave_hour  anomaly anomaly_label  
 0   2025-01-01 20:45:46.692   346.279          20        1        Normal  
 1   2

In [22]:
create_burglary_message(detection[0], detection[1])

'🚔 **Burglary Alert!** 🚔\nPotential burglary detected with the following details:\n\n🔹 **From:** livingroombedarea ➡️ **To:** kitchen\n🔸 **Leave Time:** 2025-01-02 17:09:06\n🔸 **Enter Time:** 2025-01-02 17:36:37\n\n⚠️ Please check the premises immediately. ⚠️'

In [25]:
is_burglary, msg = detect_burglary(24*7, 24*7, 1800)
if is_burglary:
    send_todo("🏠🚔 Burglary Alert! 🏠🔐", msg, 2)

Model version 1 loaded from MinIO successfully.
ToDo item saved successfully.
Response: None
